Credits: this code and documentation was adapted from Paul Butler's [sklearn-pandas](https://github.com/paulgb/sklearn-pandas)

# DataFrames

It is possible to train models based on dataframes, but they need to be converted to arrays first. `DataFrameMapper` is used to specify how this conversion proceeds. For example, PCA might be applied to some numerical dataframe columns, and one-hot-encoding to a categorical column.

## Transformation Mapping

In [ ]:
using ScikitLearn
using DataFrames: DataFrame, NA, DataArray
using DataArrays: @data
@sk_import preprocessing: (LabelBinarizer, StandardScaler)

data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])

### Map the Columns to Transformations

The mapper takes a list of pairs. The first is a column name from the DataFrame, or a list containing one or multiple columns (we will see an example with multiple columns later). The second is an object which will perform the transformation which will be applied to that column:

Note: `ScikitLearn.DataFrameMapper` won't be available until `DataFrames` is imported

In [ ]:
mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())]);

The difference between specifying the column selector as :column (as a single symbol) and [:column] (as a list with one element) is the shape of the array that is passed to the transformer. In the first case, a one dimensional array with be passed, while in the second case it will be a 2-dimensional array with one column, i.e. a column vector.

### Test the Transformation

We can use the `fit_transform!` shortcut to both fit the model and see what transformed data looks like. In this and the other examples, output is rounded to two digits with `round` to account for rounding errors on different hardware:

In [ ]:
round(fit_transform!(mapper, copy(data)), 2)

Note that the first three columns are the output of the LabelBinarizer (corresponding to `cat`, `dog`, and `fish`
 respectively) and the fourth column is the standardized value for the number of children. In general, the columns are ordered according to the order given when the DataFrameMapper is constructed.

Now that the transformation is trained, we confirm that it works on new data:

In [ ]:
sample = DataFrame(pet= ["cat"], children= [5.])
round(transform(mapper, sample), 2)

### Transform Multiple Columns

Transformations may require multiple input columns. In these cases, the column names can be specified in a list:

In [ ]:
@sk_import decomposition: PCA
mapper2 = DataFrameMapper([([:children, :salary], PCA(1))]);

Now running `fit_transform!` will run PCA on the `children` and `salary` columns and return the first principal component:

In [ ]:
round(fit_transform!(mapper2, copy(data)), 1)

### Multiple transformers for the same column

Multiple transformers can be applied to the same column specifying them in a list:

In [ ]:
@sk_import preprocessing: Imputer
mapper3 = DataFrameMapper([([:age], [Imputer()])]; NA2NaN=true)
data_3 = DataFrame(age= @data([1, NA, 3]))
fit_transform!(mapper3, data_3)

### Columns that don't need any transformation

Only columns that are listed in the `DataFrameMapper` are kept. To keep a column but don't apply any transformation to it, use `nothing` as transformer:

In [ ]:
mapper3 = DataFrameMapper([
     (:pet, LabelBinarizer()),
     (:children, nothing)])
round(fit_transform!(mapper3, copy(data)))

## Cross-validation

Now that we can combine features from a DataFrame, we may want to use cross-validation to see whether our model works.

In [ ]:
@sk_import linear_model: LinearRegression

pipe = Pipelines.Pipeline([
     (:featurize, mapper),
     (:lm, LinearRegression())])
round(CrossValidation.cross_val_score(pipe, data, data[:salary]), 2)